In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("gopalbhattrai/pascal-voc-2012-dataset")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

2025-08-28 02:58:48.822414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/garvthakral/miniconda3/envs/yolo_gpu/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [44]:
# !ls /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages

In [49]:
# !cat /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/ImageSets/Layout/train.txt
!ls /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml

/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml


In [2]:
import xml.etree.ElementTree as ET

tree = ET.parse('/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml')
root = tree.getroot()

for obj in root.findall('object'):
    cls = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)
    print(cls, xmin, ymin, xmax, ymax)

    # Optional: parse parts if needed
    for part in obj.findall('part'):
        part_name = part.find('name').text
        part_box = part.find('bndbox')
        pxmin = int(part_box.find('xmin').text)
        pymin = int(part_box.find('ymin').text)
        pxmax = int(part_box.find('xmax').text)
        pymax = int(part_box.find('ymax').text)
        print('   Part:', part_name, pxmin, pymin, pxmax, pymax)


person 174 101 349 351
   Part: head 169 104 209 146
   Part: hand 278 210 297 233
   Part: foot 273 333 297 354
   Part: foot 319 307 340 326


In [3]:
import supervision as sv
import cv2
import numpy as np

def infer(imagePath,coordinates = None):
    image = cv2.imread(imagePath)
    
    if coordinates == None:
        return image
    
    detections = sv.Detections(
        xyxy=np.array([coordinates]),
        class_id=np.array([0]),
        confidence=np.array([0.94])
    )
    
    # Create annotator using current API
    annotator = sv.BoxAnnotator()  # <- new name
    annotated_frame = annotator.annotate(scene=image.copy(), detections=detections)
    
    return annotated_frame

ModuleNotFoundError: No module named 'supervision'

In [303]:
# sv.plot_image(infer("/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages/2007_000027.jpg",coordinates= [174, 101, 349, 351]))

In [4]:
id_to_label_map = {
    0: "background",1: "aeroplane",2: "bicycle",3: "bird",4: "boat",
    5: "bottle",6: "bus",7: "car",8: "cat",
    9: "chair",10: "cow",11: "diningtable",12: "dog",
    13: "horse",14: "motorbike",15: "person",16: "pottedplant",
    17: "sheep",18: "sofa",19: "train",20: "tvmonitor"
}
label_to_id_map = {v:k for k,v in id_to_label_map.items()}

In [5]:
def loss_function(y_true,y_pred):
    loss = tf.reduce_sum(tf.square(tf.subtract(y_true,y_pred)))
    return loss

In [6]:
with open('/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/ImageSets/Main/train.txt','r') as f:
    reader = f.read()
train_arr = reader.split('\n')
train_dataset = tf.data.Dataset.from_tensor_slices(train_arr)
with open('/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/ImageSets/Main/val.txt','r') as f:
    reader = f.read()
val_arr = reader.split('\n')
val_dataset = tf.data.Dataset.from_tensor_slices(val_arr)

I0000 00:00:1756330135.444879   13929 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4149 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [7]:
import xml.etree.ElementTree as ET
def preprocess(image_id, img_dir, ann_dir):
    img_path = tf.strings.join([img_dir, "/", image_id, ".jpg"])
    xml_path = tf.strings.join([ann_dir, "/", image_id, ".xml"])

    def _xmlparser(xml_path_str):
        import xml.etree.ElementTree as ET
        xml_path_str = xml_path_str.numpy().decode()  # convert bytes to str
        tree = ET.parse(xml_path_str)
        root = tree.getroot()
        nameList, bndBoxList = [], []
        for x in root.findall('object'):
            nameList.append(x.find('name').text)
            bndBoxList.append([int(y.text) for y in x.find('bndbox')])
        # only take first object for simplicity
        one_hot_class = tf.one_hot(label_to_id_map[nameList[0]], depth=20)
        temp = tf.constant([1] + bndBoxList[0], dtype=tf.float32)
        y_label = tf.concat([temp, one_hot_class], axis=0)
        return y_label

    y_label = tf.py_function(func=_xmlparser, inp=[xml_path], Tout=tf.float32)
    y_label.set_shape([25])  # objectness + bbox + 20 classes

    # Load and preprocess image
    img_raw = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize_with_pad(img, 448, 448)
    img = tf.cast(img, tf.float32) / 255.0

    return img, y_label


In [8]:
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf

tree = ET.parse('/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000033.xml')
root = tree.getroot()

nameList = []
bndBoxList = []

# Get original image width and height
size = root.find('size')
W = int(size.find('width').text)
H = int(size.find('height').text)

# Extract objects + bounding boxes
for x in root.findall('object'):
    name = x.find('name').text
    bndbox = x.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)

    # normalize using W, H
    xmin /= W
    xmax /= W
    ymin /= H
    ymax /= H

    nameList.append(name)
    bndBoxList.append([xmin, ymin, xmax, ymax])

bndBoxList = np.array(bndBoxList, dtype=np.float32)
print("Normalized bounding boxes:\n", bndBoxList)

# Example: building label tensor for the first box
for x in range(len(nameList)):
    one_hot_class = tf.one_hot(label_to_id_map[nameList[x]], depth=20)
    temp = tf.constant([1] + bndBoxList[x].tolist(), dtype=tf.float32)  # [objectness, xmin, ymin, xmax, ymax]
    y_label = tf.concat([temp, one_hot_class], axis=0)
    print("Final label vector:", y_label.numpy())
    break


Normalized bounding boxes:
 [[0.018      0.29234973 0.998      0.71857923]
 [0.842      0.5464481  0.964      0.61748636]
 [0.65       0.5136612  0.822      0.60928965]]
Final label vector: [1.         0.018      0.29234973 0.998      0.71857923 0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [9]:
img_dir = '/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages'
ann_dir = '/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations'
# for x in train_dataset.map(lambda x:preprocess(x,img_dir,ann_dir)).take(1):
#     print(x)

In [117]:
# image_dataset = tf.raw_ops.LoadDataset(
#     path ="/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages",   
# )

TypeError: load_dataset() missing 4 required positional arguments: 'reader_func_other_args', 'output_types', 'output_shapes', and 'reader_func'

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, LeakyReLU, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def TinyYOLO_Light(input_shape=(448, 448, 3), num_classes=20):
    inputs = Input(shape=input_shape)

    # Layer 1
    X = Conv2D(16, (3,3), strides=1, padding='same')(inputs)
    X = BatchNormalization()(X)
    X = LeakyReLU(0.1)(X)
    X = MaxPooling2D((2,2), strides=2)(X)

    # Layer 2
    X = Conv2D(32, (3,3), padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(0.1)(X)
    X = MaxPooling2D((2,2), strides=2)(X)

    # Layer 3
    X = Conv2D(64, (3,3), padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(0.1)(X)
    X = MaxPooling2D((2,2), strides=2)(X)

    # Layer 4
    X = Conv2D(128, (3,3), padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(0.1)(X)
    X = MaxPooling2D((2,2), strides=2)(X)

    # Layer 5
    X = Conv2D(256, (3,3), padding='same')(X)
    X = BatchNormalization()(X)
    X = LeakyReLU(0.1)(X)
    X = MaxPooling2D((2,2), strides=2)(X)

    # Global Pooling instead of Flatten
    X = GlobalAveragePooling2D()(X)

    # Dense layers for predictions
    X = Dense(512, activation='relu')(X)
    outputs = Dense(5 + num_classes, activation='sigmoid')(X)  # [objectness, bx, by, bw, bh, class probs]

    model = Model(inputs, outputs)
    return model

# Create model
model = TinyYOLO_Light()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 448, 448, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 448, 448, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 448, 448, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 448, 448, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 224, 224, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │             

 Total params: 539,001 (2.06 MB)

 Trainable params: 538,009 (2.05 MB)

 Non-trainable params: 992 (3.88 KB)

In [11]:
train_dataset_x = train_dataset.map(lambda x:preprocess(x,img_dir,ann_dir)).batch(1).prefetch(tf.data.AUTOTUNE)
model.compile(
    loss = loss_function,
    optimizer = "adam",
    metrics = ['accuracy']
)
model.fit(train_dataset_x)

2025-08-28 02:59:11.441136: I external/local_xla/xla/service/service.cc:163] XLA service 0x731794003cc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-28 02:59:11.441175: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-08-28 02:59:11.523252: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-28 02:59:11.954982: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91200


  21/5718 ━━━━━━━━━━━━━━━━━━━━ 47s 8ms/step - accuracy: 0.5933 - loss: 272272.9375

I0000 00:00:1756330155.612587   14015 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5711/5718 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4908 - loss: 291592.9062

2025-08-28 02:59:57.780639: W tensorflow/core/framework/op_kernel.cc:1855] OP_REQUIRES failed at whole_file_read_ops.cc:117 : NOT_FOUND: /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages/.jpg; No such file or directory
2025-08-28 02:59:57.783028: W tensorflow/core/framework/op_kernel.cc:1842] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/Annotations/.xml'
Traceback (most recent call last):

  File "/home/garvthakral/miniconda3/envs/yolo_gpu/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/garvthakral/miniconda3/envs/yolo_gpu/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 145, in __call__
    outputs 

NotFoundError: Graph execution error:

Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
Detected at node ReadFile defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) NOT_FOUND:  /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages/.jpg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) NOT_FOUND:  /home/garvthakral/.cache/kagglehub/datasets/gopalbhattrai/pascal-voc-2012-dataset/versions/1/VOC2012_train_val/VOC2012_train_val/JPEGImages/.jpg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_multi_step_on_iterator_5068]

In [12]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
#!/usr/bin/env python3
"""
TensorFlow GPU Debug Script
Comprehensive check for GPU detection issues
"""

import os
import sys
import subprocess
import platform

def print_section(title):
    print(f"\n{'='*50}")
    print(f" {title}")
    print(f"{'='*50}")

def run_command(cmd):
    """Run shell command and return output"""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        return result.stdout.strip(), result.stderr.strip()
    except Exception as e:
        return "", str(e)

def check_system_info():
    print_section("SYSTEM INFORMATION")
    print(f"OS: {platform.system()} {platform.release()}")
    print(f"Python: {sys.version}")
    print(f"Architecture: {platform.machine()}")
    
def check_nvidia_driver():
    print_section("NVIDIA DRIVER CHECK")
    
    stdout, stderr = run_command("nvidia-smi")
    if stdout:
        print("✅ NVIDIA driver detected!")
        print("GPU Information:")
        lines = stdout.split('\n')
        for line in lines:
            if 'NVIDIA-SMI' in line or 'GeForce' in line or 'Tesla' in line or 'RTX' in line or 'GTX' in line:
                print(f"  {line.strip()}")
    else:
        print("❌ NVIDIA driver not found!")
        print(f"Error: {stderr}")
        return False
    
    return True

def check_cuda():
    print_section("CUDA CHECK")
    
    # Check nvcc
    stdout, stderr = run_command("nvcc --version")
    if stdout:
        print("✅ CUDA Compiler (nvcc) found!")
        for line in stdout.split('\n'):
            if 'release' in line.lower():
                print(f"  {line.strip()}")
    else:
        print("❌ CUDA Compiler (nvcc) not found!")
        print("Try: export PATH=/usr/local/cuda/bin:$PATH")
    
    # Check CUDA installation path
    cuda_paths = [
        "/usr/local/cuda",
        "/opt/cuda",
        "/usr/lib/cuda"
    ]
    
    for path in cuda_paths:
        if os.path.exists(path):
            print(f"✅ CUDA installation found at: {path}")
            break
    else:
        print("❌ CUDA installation directory not found!")
    
    # Check environment variables
    print("\nCUDA Environment Variables:")
    cuda_vars = ["CUDA_HOME", "CUDA_PATH", "CUDA_ROOT", "PATH", "LD_LIBRARY_PATH"]
    for var in cuda_vars:
        value = os.environ.get(var, "Not set")
        if "cuda" in value.lower():
            print(f"  {var}: {value}")

def check_cudnn():
    print_section("CUDNN CHECK")
    
    # Check common cuDNN locations
    cudnn_locations = [
        "/usr/local/cuda/include/cudnn.h",
        "/usr/include/cudnn.h",
        "/opt/cuda/include/cudnn.h"
    ]
    
    for location in cudnn_locations:
        if os.path.exists(location):
            print(f"✅ cuDNN header found at: {location}")
            
            # Try to get cuDNN version
            stdout, _ = run_command(f"cat {location} | grep CUDNN_MAJOR -A 2")
            if stdout:
                print("cuDNN Version:")
                for line in stdout.split('\n')[:3]:
                    if 'CUDNN_' in line:
                        print(f"  {line.strip()}")
            break
    else:
        print("❌ cuDNN headers not found!")
        
    # Check for cuDNN library
    stdout, _ = run_command("find /usr -name '*cudnn*' 2>/dev/null | head -5")
    if stdout:
        print("cuDNN libraries found:")
        for line in stdout.split('\n'):
            print(f"  {line}")

def check_tensorflow():
    print_section("TENSORFLOW CHECK")
    
    try:
        import tensorflow as tf
        print(f"✅ TensorFlow version: {tf.__version__}")
        
        # Check if it's GPU-enabled build
        print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
        print(f"Built with GPU support: {tf.test.is_built_with_gpu_support()}")
        
    except ImportError:
        print("❌ TensorFlow not installed!")
        return False
    
    return True

def check_gpu_detection():
    print_section("GPU DETECTION TEST")
    
    try:
        import tensorflow as tf
        
        # List physical devices
        physical_devices = tf.config.list_physical_devices()
        print("All Physical Devices:")
        for device in physical_devices:
            print(f"  {device}")
        
        # List GPU devices specifically
        gpu_devices = tf.config.list_physical_devices('GPU')
        print(f"\nGPU Devices Found: {len(gpu_devices)}")
        
        if gpu_devices:
            print("✅ GPU(s) detected by TensorFlow!")
            for i, gpu in enumerate(gpu_devices):
                print(f"  GPU {i}: {gpu}")
                
                # Get GPU details
                try:
                    gpu_details = tf.config.experimental.get_device_details(gpu)
                    print(f"    Details: {gpu_details}")
                except Exception as e:
                    print(f"    Details unavailable: {e}")
            
            return True
        else:
            print("❌ No GPU devices found by TensorFlow!")
            return False
            
    except Exception as e:
        print(f"❌ Error checking GPU detection: {e}")
        return False

def check_memory_growth():
    print_section("GPU MEMORY TEST")
    
    try:
        import tensorflow as tf
        
        gpu_devices = tf.config.list_physical_devices('GPU')
        if not gpu_devices:
            print("❌ No GPUs to test memory on!")
            return
        
        print("Setting memory growth...")
        for gpu in gpu_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print("✅ Memory growth configured!")
        
        # Simple computation test
        print("Running simple GPU computation test...")
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
            b = tf.constant([[1.0, 1.0], [0.0, 1.0]])
            c = tf.matmul(a, b)
            print(f"✅ GPU computation successful! Result: {c.numpy()}")
            
    except Exception as e:
        print(f"❌ GPU memory/computation test failed: {e}")

def check_versions_compatibility():
    print_section("VERSION COMPATIBILITY CHECK")
    
    try:
        import tensorflow as tf
        
        print("TensorFlow-CUDA-cuDNN Compatibility:")
        tf_version = tf.__version__
        print(f"TensorFlow: {tf_version}")
        
        # Common compatible versions
        compatibility = {
            "2.15.0": {"cuda": "12.2", "cudnn": "8.9"},
            "2.14.0": {"cuda": "11.8", "cudnn": "8.7"},
            "2.13.0": {"cuda": "11.8", "cudnn": "8.6"},
            "2.12.0": {"cuda": "11.8", "cudnn": "8.6"},
            "2.11.0": {"cuda": "11.2", "cudnn": "8.1"},
        }
        
        major_minor = '.'.join(tf_version.split('.')[:2]) + '.0'
        if major_minor in compatibility:
            req_cuda = compatibility[major_minor]['cuda']
            req_cudnn = compatibility[major_minor]['cudnn']
            print(f"Recommended CUDA: {req_cuda}")
            print(f"Recommended cuDNN: {req_cudnn}")
        else:
            print("Check TensorFlow docs for your version's requirements")
            
    except ImportError:
        print("TensorFlow not available for version check")

def run_diagnostic_commands():
    print_section("ADDITIONAL DIAGNOSTICS")
    
    commands = {
        "CUDA Runtime Version": "cat /usr/local/cuda/version.txt",
        "GPU Memory Info": "nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv",
        "NVIDIA Driver Version": "cat /proc/driver/nvidia/version",
        "Python Site Packages": f"python -c \"import site; print(site.getsitepackages())\"",
    }
    
    for name, cmd in commands.items():
        stdout, stderr = run_command(cmd)
        print(f"\n{name}:")
        if stdout:
            print(f"  {stdout}")
        elif stderr:
            print(f"  Error: {stderr}")
        else:
            print(f"  Command not available: {cmd}")

def suggest_fixes():
    print_section("COMMON FIXES")
    
    fixes = [
        "1. Install/Update NVIDIA drivers:",
        "   sudo apt update && sudo apt install nvidia-driver-XXX",
        "",
        "2. Install CUDA toolkit:",
        "   wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin",
        "   sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600",
        "   # Download and install CUDA from NVIDIA website",
        "",
        "3. Install TensorFlow with GPU support:",
        "   pip install tensorflow[and-cuda]  # TF 2.12+",
        "   # or pip install tensorflow-gpu  # Older versions",
        "",
        "4. Set environment variables:",
        "   export PATH=/usr/local/cuda/bin:$PATH",
        "   export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH",
        "",
        "5. Docker option (if all else fails):",
        "   docker run --gpus all -it tensorflow/tensorflow:latest-gpu python",
        "",
        "6. Restart system after driver installation!",
    ]
    
    for fix in fixes:
        print(fix)

def main():
    print("🔍 TensorFlow GPU Debug Script")
    print("This will help diagnose GPU detection issues")
    
    # Run all checks
    check_system_info()
    
    has_nvidia = check_nvidia_driver()
    check_cuda()
    check_cudnn()
    
    has_tf = check_tensorflow()
    if has_tf:
        has_gpu = check_gpu_detection()
        if has_gpu:
            check_memory_growth()
        check_versions_compatibility()
    
    run_diagnostic_commands()
    suggest_fixes()
    
    print_section("SUMMARY")
    print("If you're still having issues:")
    print("1. Check TensorFlow installation docs")
    print("2. Consider using conda for easier dependency management")
    print("3. Try the official TensorFlow Docker images")
    print("4. Post the output of this script when asking for help")

if __name__ == "__main__":
    main()

🔍 TensorFlow GPU Debug Script
This will help diagnose GPU detection issues

 SYSTEM INFORMATION
OS: Linux 6.11.0-29-generic
Python: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
Architecture: x86_64

 NVIDIA DRIVER CHECK
✅ NVIDIA driver detected!
GPU Information:
  | NVIDIA-SMI 570.133.07             Driver Version: 570.133.07     CUDA Version: 12.8     |
  |   0  NVIDIA GeForce RTX 3060 ...    Off |   00000000:01:00.0 Off |                  N/A |

 CUDA CHECK
✅ CUDA Compiler (nvcc) found!
  Cuda compilation tools, release 12.8, V12.8.61
✅ CUDA installation found at: /usr/local/cuda

CUDA Environment Variables:
  PATH: /home/garvthakral/miniconda3/envs/yolo_gpu/bin:/usr/local/cuda-12.8/bin:/home/garvthakral/miniconda3/condabin:/home/garvthakral/.local/bin:/home/garvthakral/.pyenv/plugins/pyenv-virtualenv/shims:/home/garvthakral/.pyenv/shims:/home/garvthakral/.pyenv/bin:/home/garvthakral/.local/bin:/home/garvthakral/.nvm/versions/node/v20.12.2/bin:/home/garvthakral/.cargo/bin:/home

2025-08-28 02:10:12.023065: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/garvthakral/miniconda3/envs/yolo_gpu/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


✅ TensorFlow version: 2.20.0
Built with CUDA: True
Built with GPU support: True

 GPU DETECTION TEST
All Physical Devices:
  PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

GPU Devices Found: 1
✅ GPU(s) detected by TensorFlow!
  GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
    Details: {'compute_capability': (8, 6), 'device_name': 'NVIDIA GeForce RTX 3060 Laptop GPU'}

 GPU MEMORY TEST
Setting memory growth...
✅ Memory growth configured!
Running simple GPU computation test...


I0000 00:00:1756327216.391939   16882 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4149 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


✅ GPU computation successful! Result: [[1. 3.]
 [3. 7.]]

 VERSION COMPATIBILITY CHECK
TensorFlow-CUDA-cuDNN Compatibility:
TensorFlow: 2.20.0
Check TensorFlow docs for your version's requirements

 ADDITIONAL DIAGNOSTICS

CUDA Runtime Version:
  Error: cat: /usr/local/cuda/version.txt: No such file or directory

GPU Memory Info:
  memory.total [MiB], memory.used [MiB], memory.free [MiB]
6144 MiB, 145 MiB, 5672 MiB

NVIDIA Driver Version:
  NVRM version: NVIDIA UNIX x86_64 Kernel Module  570.133.07  Fri Mar 14 13:12:07 UTC 2025
GCC version:  gcc version 14.2.0 (Ubuntu 14.2.0-4ubuntu2)

Python Site Packages:
  ['/home/garvthakral/miniconda3/envs/yolo_gpu/lib/python3.11/site-packages']

 COMMON FIXES
1. Install/Update NVIDIA drivers:
   sudo apt update && sudo apt install nvidia-driver-XXX

2. Install CUDA toolkit:
   wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
   sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pi

In [ ]:
!pip uninstall -y tensorflow

!conda install -c conda-forge cudatoolkit cudnn
!conda install -c conda-forge tensorflow-gpu